## Snapista Demo notebook - Using band maths, setting up operators, creating and running a graph...

In [1]:
# Some information...

import os
import sys
print("Python version: " + sys.version)

import sysconfig
print("Location of esa_snappy package: " + sysconfig.get_paths()['purelib'] + os.sep + "esa_snappy")

Python version: 3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]
Location of esa_snappy package: D:\olaf\miniconda3_py311\Lib\site-packages\esa_snappy


In [2]:
# Import esa_snappy and snapista...

import esa_snappy
import snapista

from snapista import Graph
from snapista import Operator
from snapista import read_file
from snapista import TargetBand
from snapista import TargetBandDescriptors

In [3]:
# Set up a graph...

try:
    g = Graph()
except Exception as ex:
    print("Cannot set up Snapista Graph():", ex)

In [4]:
# Add 'Read' operator to the graph for reading example input...

g.add_node(operator=Operator("Read", file='./data/subset_0_of_S3A_OL_1_ERR____20160701T092853_20160701T093053_20170919T102725_0119_006_036______MR1_R_NT_002.nc'), node_id="Read")


In [5]:
# Set up a 'BandMaths' operator to compute a simple NDVI. Set up a target band for NDVI result...

band_maths = Operator('BandMaths')

ndvi = TargetBand(name='ndvi', expression='(Oa08_radiance - Oa17_radiance)/(Oa08_radiance + Oa17_radiance)')
                         
band_maths.targetBandDescriptors = TargetBandDescriptors([ndvi])

In [6]:
# Add 'BandMaths' operator to the graph...

g.add_node(operator=band_maths, node_id="BandMaths" ,source='Read')

targetBandDescriptors <snapista.target_band_descriptors.TargetBandDescriptors object at 0x000001B75A9730D0>
Instance TargetBandDescriptors: True
Instance Aggregators: False
Instance BinningOutputBands: False
Instance BinningVariables: False
Instance str: False


In [7]:
# Set up a 'Merge' operator to merge input with NDVI result...

merge = Operator("BandMerge", source=["Read", "BandMaths"])

merge

Operator('BandMerge', sourceBandNames='None', geographicError='1.0E-5f', source='['Read', 'BandMaths']')

In [8]:
# Add 'Merge' operator to the graph...

g.add_node(operator=merge, node_id="Merge", source=["Read", "BandMaths"])

In [9]:
# Set up a 'Subset' operator to extract just three bands...

subset = Operator('Subset', bandNames="Oa08_radiance,Oa17_radiance,ndvi")

subset

Operator('Subset', bandNames='Oa08_radiance,Oa17_radiance,ndvi', tiePointGridNames='None', region='None', referenceBand='None', geoRegion='None', subSamplingX='1', subSamplingY='1', fullSwath='false', copyMetadata='false')

In [10]:
# Add 'Subset' operator to the graph...

g.add_node(operator=subset, node_id="Subset", source='Merge')

In [11]:
# Set up a 'Write' operator to write final target product with the three bands...

write = Operator("Write")

write

Operator('Write', file='None', formatName='BEAM-DIMAP', deleteOutputOnFailure='true', writeEntireTileRows='false', clearCacheAfterRowWrite='false')

In [12]:
# Add 'Write' operator to the graph. Define a NetCDF output file...

g.add_node(operator=Operator("Write", file='./data/snapista_demo_bandmaths_result.nc', formatName='NetCDF4-BEAM'), node_id="Write", source='Subset')

In [13]:
# View graph in XML representation...

g.view()

<graph>
  <version>1.0</version>
  <node id="Read">
    <operator>Read</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <bandNames/>
      <copyMetadata>true</copyMetadata>
      <file>./data/subset_0_of_S3A_OL_1_ERR____20160701T092853_20160701T093053_20170919T102725_0119_006_036______MR1_R_NT_002.nc</file>
      <formatName/>
      <geometryRegion/>
      <maskNames/>
      <pixelRegion/>
      <useAdvancedOptions>false</useAdvancedOptions>
    </parameters>
  </node>
  <node id="BandMaths">
    <operator>BandMaths</operator>
    <sources>
      <sourceProduct refid="Read"/>
    </sources>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <targetBands>
        <targetBand>
          <name>ndvi</name>
          <expression>(Oa08_radiance - Oa17_radiance)/(Oa08_radiance + Oa17_radiance)</expression>
          <type>float32</type>
          <description/>
          <unit/>
          <no_data_value>NaN</no_data_value>
    

In [14]:
# Save graph as XML file, then run it...

g.save_graph('./data/snapista_demo_bandmaths.xml')
g.run()

Processing the graph, this may take a while. Please wait...
Executing processing graph
15%30%45%60%...70%..90% done.
SEVERE: org.esa.snap.runtime.Engine: Failed to modify class loader field 'usr_paths'
java.lang.NoSuchMethodException: java.lang.ClassLoader.initializePath(java.lang.String)
	at java.base/java.lang.Class.getDeclaredMethod(Class.java:2848)
	at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:286)
	at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:265)
	at org.esa.snap.runtime.Engine.<init>(Engine.java:46)
	at org.esa.snap.runtime.Engine.start(Engine.java:117)
	at org.esa.snap.runtime.Engine.start(Engine.java:90)
	at org.esa.snap.runtime.Launcher.run(Launcher.java:51)
	at org.esa.snap.runtime.Launcher.main(Launcher.java:31)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at com.exe4j.runtime.LauncherEngine.launch(LauncherEngine.java

0